# Runner for Experiment 2

## Experiment

In [1]:
from qiskit import assemble
from qiskit.transpiler import CouplingMap
from qiskit.transpiler.passes import CSPLayout, CspRbsLayout, DenseLayout, \
                                     NoiseAdaptiveLayout, SabreLayout
from qiskit.providers.aer import QasmSimulator
from qiskit.providers.aer.noise import NoiseModel

from exp.custom_passmanager import custom_pass_manager

In [2]:
def exp2_evaluate(circuit, layout_method, backend, ideal=True, shots=1, seed=None, pass_reps=10):
    
    coupling_map = CouplingMap(backend.configuration().coupling_map)
    backend_properties = backend.properties()

    if layout_method == 'csplayout':
        layout = CSPLayout(coupling_map, seed=seed, call_limit=None, time_limit=None)
    elif layout_method == 'csprbslayout':
        layout = CspRbsLayout(coupling_map, seed=seed, call_limit=None, time_limit=None, \
                              solution_limit=20, iteration_limit=10, backend_prop=backend_properties)
    elif layout_method == 'dense':
        layout = DenseLayout(coupling_map, None if ideal else backend_properties)
    elif layout_method == 'noise_adaptive':
        layout = NoiseAdaptiveLayout(backend_properties)
    elif layout_method == 'sabre':
        layout = SabreLayout(coupling_map, max_iterations=1, seed=seed)
    else:
        raise Exception('layout_method unknown %s' % layout_method)

    if ideal:
        simulator = QasmSimulator(configuration=backend.configuration(),
                                  method='stabilizer')
    else:
        noise_model = NoiseModel.from_backend(backend, thermal_relaxation=False)
        simulator = QasmSimulator(configuration=backend.configuration(),
                                  noise_model=noise_model,
                                  method='stabilizer')

    passmanager = custom_pass_manager(backend, layout, seed=seed)

    times = {}
    count_ops_after_map = {}

    def callback(**kwargs):
        times[kwargs['pass_'].name()] = times.get(kwargs['pass_'].name(), 0) + kwargs['time']
        if kwargs['pass_'].name() == 'StochasticSwap':
            count_ops_after_map.update(kwargs['dag'].count_ops())

    for rep in range(pass_reps):
        transpiled = passmanager.run(circuit, callback=callback)

    needed_swaps = 0 if passmanager.property_set['is_swap_mapped'] else count_ops_after_map['swap']

    qobj = assemble(transpiled, backend, shots=shots, seed_simulator=seed)
    result = simulator.run(qobj).result()
    counts = result.get_counts()
    time = times[layout.name()] / pass_reps

    return counts, time, needed_swaps

## Runner

In [3]:
import pandas as pd
from qiskit.test.mock.backends import FakeManhattan
from qiskit.providers.aer import QasmSimulator

from exp.circuit_generators import graphstate_complete
from exp.write_to_df import write_to_dataframe

In [4]:
def exp2_run(layout_method, exp_path, num_samples=100, num_qubits=12, num_shots=8192):
    backend = FakeManhattan()
    circuit = graphstate_complete(num_qubits)

    df = pd.DataFrame(columns=["seed", "state", "ideal_count", "noise_count", \
                           "ideal_time", "noise_time", "ideal_swaps_needed", "noise_swaps_needed"])
    for num_index in range(num_samples):
        seed = num_index
        ideal_counts, ideal_time, swaps_needed_ideal = exp2_evaluate(
            circuit, layout_method, backend=backend, ideal=True, shots=num_shots, seed=seed)
        noise_counts, noise_time, swaps_needed_noise = exp2_evaluate(
            circuit, layout_method, backend=backend, ideal=False, shots=num_shots, seed=seed)
        total_result = {'seed': seed,
                        'ideal_counts': ideal_counts,
                        'noise_counts': noise_counts,
                        'ideal_time': ideal_time,
                        'noise_time': noise_time,
                        'ideal_swaps_needed': swaps_needed_ideal,
                        'noise_swaps_needed': swaps_needed_noise}

        df_loc = write_to_dataframe(total_result)
        df = df.append(df_loc)

    df.to_csv(exp_path + f'exp2_{layout_method}.csv')

## Execution

In [5]:
layout_methods = [
    'csprbslayout',
    'dense',
    'noise_adaptive',
    'sabre'
]
exp_path = "./data/"
num_samples = 1
for method in layout_methods:
    exp2_run(method, exp_path, num_samples=num_samples)

KeyError: 'swap'